In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [28]:
# Get HTML page with the desired_data from wikipedia
desired_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Create BeaultifulSoup object
soup = BeautifulSoup(desired_data, "html.parser")
                     
#print(soup.prettify())

In [29]:
# Find the desired_data under class 'Wikitable Sortable'.
wikip_table = soup.find('table', {'class':'wikitable sortable'})
#print(wikip_table.prettify())



In [30]:
# Collect rows from table
wikip_table_rows = wikip_table.findAll('tr')
#print(wikip_table_rows)

In [31]:
# Initialize df
sol = [] 
for tr in wikip_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    #print(row)
    if row:
        if (row[1]!='Not assigned'):
            if (row[2]=='Not assigned'):
                row[2]=row[1]
            sol.append(row)
#print('\n')            
#print(sol)

In [135]:
# Reformatting to join neighborhoods from the same borough
for tr in wikip_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if (row[1]!='Not assigned'):
            if (row[2]=='Not assigned'):
                row[2]=row[1]
            for j in range(0, len(sol)):
                if (row[0]==sol[j][0]):
                    if (row[2]!=sol[j][2]):
                        if row[2] not in sol[j][2]:
                            sol[j][2] = sol[j][2]+','+row[2]
#print(res)                        
# Set Pandas DataFrame columns names
df = pd.DataFrame(sol, columns=["Postcode", "Borough", "Neighborhood"])
# Drop duplicate rows:
df = df.drop_duplicates(subset='Postcode', keep='first', inplace=False)
df= df.rename(columns={'Postcode': 'Postal Code'})
df.head()
print(df.shape)

(103, 3)


In [132]:
import pandas as pd
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
data = pd.read_csv('Geospatial_Coordinates.csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [138]:
result = pd.merge(df, data, on='Postal Code')
print(result.head())

  Postal Code           Borough                     Neighborhood   Latitude  Longitude
0         M3A        North York                        Parkwoods  43.753259 -79.329656
1         M4A        North York                 Victoria Village  43.725882 -79.315572
2         M5A  Downtown Toronto         Harbourfront,Regent Park  43.654260 -79.360636
3         M6A        North York  Lawrence Heights,Lawrence Manor  43.718518 -79.464763
4         M7A      Queen's Park                     Queen's Park  43.662301 -79.389494


# **Import Libraries**

In [136]:

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    numpy-1.14.2               |   py36hdbf6ddf_0         4.0 MB
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    pandas-0.23.4              |   py36hf8a1672_0        27.8 MB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    folium-0.5.0         

In [160]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

## Filter our DataFrame - We will only work with Boroughs with 'Toronto' in the name. 

In [139]:
result2 = result.copy()
result_filtered = result2.query("Borough=='West Toronto' or Borough=='Central Toronto' or Borough=='Downtown Toronto' or Borough=='East Toronto' ").reset_index(drop=True)
result_filtered

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


## Create Folium Map of Toronto

In [141]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.665860, -79.383160], zoom_start=12)

# add markers to map
for lat, lng, label in zip(result_filtered['Latitude'], result_filtered['Longitude'], result_filtered['Neighborhood']):
    label = folium.Popup()
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

**Define Foursquare Credentials and Version** 

In [143]:
CLIENT_ID = 'W1ZY1YZGYIMN2EECTKEMNRPHBOR1Y0KHJUQZJK305OQ2Y0PZ' # your Foursquare ID
CLIENT_SECRET = '04JIKKMAB1QY1GQOBLCEABQZMVK0EA3YD4QKTL4XGCI12HAZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1ZY1YZGYIMN2EECTKEMNRPHBOR1Y0KHJUQZJK305OQ2Y0PZ
CLIENT_SECRET:04JIKKMAB1QY1GQOBLCEABQZMVK0EA3YD4QKTL4XGCI12HAZ


## Using Foursquare API to explore the neighborhoods.

 **getNearbyVenues() is a function made to get the top 100 venues that are in each neighborhood within a radius of 500 meters.**

 **The venue data is stored in the toronto_venues DataFrame.**


In [144]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [146]:
toronto_venues = getNearbyVenues(names=result_filtered['Neighborhood'],
                                 latitudes=result_filtered['Latitude'],
                                 longitudes=result_filtered['Longitude'])
toronto_venues.head()

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [147]:
# Let's check the size of the resulting dataframe
print(toronto_venues.shape)

(1691, 7)


In [148]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business reply mail Processing Centre969 Eastern,14,14,14,14,14,14
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",12,12,12,12,12,12
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",99,99,99,99,99,99
Christie,16,16,16,16,16,16


In [151]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


### Analyze each neighborhood.


In [152]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.tail()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,...,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
1686,0,0,0,0,0,...,0,0,0,0,0
1687,0,0,0,0,0,...,0,0,0,0,0
1688,0,0,0,0,0,...,0,0,0,0,0
1689,0,0,0,0,0,...,0,0,0,0,0
1690,0,0,0,0,0,...,0,0,0,0,0


In [153]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,...,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
3,Business reply mail Processing Centre969 Eastern,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012048,0.000000,0.000000,0.000000,...,0.000000,0.012048,0.000000,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,...,0.050505,0.010101,0.000000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011905,0.011905,0.011905,0.000000,...,0.011905,0.000000,0.011905,0.011905,0.00


In [163]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [155]:
#Let's put that into a pandas dataframe¶
#First, let's write a function to sort the venues in descending order.


import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,...,Hotel,Restaurant,Bar,Gym,Clothing Store
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,...,Bakery,Pub,Farmers Market,Steakhouse,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,...,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Restaurant
3,Business reply mail Processing Centre969 Eastern,Recording Studio,Garden,Skate Park,Light Rail Station,...,Fast Food Restaurant,Brewery,Burrito Place,Restaurant,Auto Workshop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,...,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Doner Restaurant
5,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Bakery,Pizza Place,...,Italian Restaurant,Café,Indian Restaurant,Market,Playground
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Burger Joint,...,Sandwich Place,Bar,Bubble Tea Shop,Chinese Restaurant,Falafel Restaurant
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,...,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Gaming Cafe
8,Christie,Café,Grocery Store,Park,Athletics & Sports,...,Diner,Restaurant,Nightclub,Convenience Store,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,...,Burger Joint,Café,Men's Store,Mediterranean Restaurant,Pub


##  Cluster Neighborhoods

In [165]:
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
#print(toronto_grouped_clustering)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [168]:
toronto_merged = result_filtered

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,...,Restaurant,Mexican Restaurant,Theater,Breakfast Spot,Shoe Store
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,...,Middle Eastern Restaurant,Tea Room,American Restaurant,Sandwich Place,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,...,Cosmetics Shop,Bakery,Cocktail Bar,Park,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,...,Bakery,Pub,Farmers Market,Steakhouse,Cheese Shop


In [171]:
# create map
map_clusters = folium.Map(location=[43.665860, -79.383160], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine cluster in Toronto

In [172]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Park,...,Restaurant,Mexican Restaurant,Theater,Breakfast Spot,Shoe Store
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,...,Middle Eastern Restaurant,Tea Room,American Restaurant,Sandwich Place,Italian Restaurant
2,Downtown Toronto,0,Coffee Shop,Restaurant,Café,...,Cosmetics Shop,Bakery,Cocktail Bar,Park,Gastropub
3,East Toronto,0,Coffee Shop,Trail,Pub,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,Downtown Toronto,0,Coffee Shop,Restaurant,Cocktail Bar,...,Bakery,Pub,Farmers Market,Steakhouse,Cheese Shop
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,...,Sandwich Place,Bar,Bubble Tea Shop,Chinese Restaurant,Falafel Restaurant
6,Downtown Toronto,0,Café,Grocery Store,Park,...,Diner,Restaurant,Nightclub,Convenience Store,Coffee Shop
7,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,...,Hotel,Restaurant,Bar,Gym,Clothing Store
8,West Toronto,0,Pharmacy,Supermarket,Bakery,...,Café,Bus Stop,Middle Eastern Restaurant,Discount Store,Brewery
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,...,Italian Restaurant,Brewery,Scenic Lookout,Restaurant,Sports Bar


In [173]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Toronto,1,Gift Shop,Breakfast Spot,Piano Bar,...,Eastern European Restaurant,Bar,Burger Joint,Dog Run,Italian Restaurant


In [174]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Playground,Women's Store,Dessert Shop,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [175]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Bus Line,Park,Swim School,...,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
27,West Toronto,3,Coffee Shop,Pizza Place,Café,...,Butcher,Food,Indie Movie Theater,Fish & Chips Shop,Bar


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]